# 문제1

### **문제 1) Tokenizer 생성하기**

**1-1. `preprocessing()`**

텍스트 전처리를 하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장을 토큰화한 결과로, nested list 형태입니다. ex) [['i', 'go', 'to', 'school'], ['i', 'like', 'pizza']]
- 조건 1: 입력된 문장에 대해서 소문자로의 변환과 특수문자 제거를 수행합니다.
- 조건 2: 토큰화는 white space 단위로 수행합니다.
    
    

**1-2. `fit()`**

어휘 사전을 구축하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: 위에서 만든 `preprocessing` 함수를 이용하여 각 문장에 대해 토큰화를 수행합니다.
- 조건 2: 각각의 토큰을 정수 인덱싱 하기 위한 어휘 사전(`self.word_dict`)을 생성합니다.
    - 주어진 코드에 있는 `self.word_dict`를 활용합니다.
    

**1-3. `transform()`**

어휘 사전을 활용하여 입력 문장을 정수 인덱싱하는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장의 정수 인덱싱으로, nested list 형태입니다. ex) [[1, 2, 3, 4], [1, 5, 6]]
- 조건 1: 어휘 사전(`self.word_dict`)에 없는 단어는 'oov'의 index로 변환합니다.

In [41]:
import string

class Tokenizer():
  def __init__(self):
    self.word_dict = {'oov': 0}
    self.fit_checker = False
  
  def preprocessing(self, sequences):
    result = []
    for s in sequences:
        new_s = s.lower().translate(str.maketrans('', '', string.punctuation))
        result.append(list(new_s.split(' ')))
    return result
  
  def fit(self, sequences):
    self.fit_checker = False
    fitted_docs = self.preprocessing(sequences)
    fitted_docs2 = list(set(sum(fitted_docs, [])))
    token_index = 1
    for t in fitted_docs2:
        self.word_dict.update({t: token_index})
        token_index+=1  
    self.fit_checker = True
    print(f"fitted word dict: {self.word_dict}")
  
  def transform(self, sequences):
    result = []
    fitted_docs = self.preprocessing(sequences)
    if self.fit_checker:
        for doc in fitted_docs:
            arr = [self.word_dict[w]  if self.word_dict[w] != None else self.word_dict['oov'] for w in doc]
            result.append(arr)
        return result
    else:
      raise Exception("Tokenizer instance is not fitted yet.")
      
  def fit_transform(self, sequences):
    self.fit(sequences)
    result = self.transform(sequences)
    return result




In [42]:
example_set = ['I go to school.', 'I LIKE pizza!']

token = Tokenizer()

print("문제 1-1")
arr = token.preprocessing(example_set)
print(f"전처리 결과값: {arr}")

print("문제 1-2")
token.fit(example_set)

result = token.transform(example_set)
print("문제 1-3")
print(f"결과값: {result}")

문제 1-1
전처리 결과값: [['i', 'go', 'to', 'school'], ['i', 'like', 'pizza']]
문제 1-2
fitted word dict: {'oov': 0, 'like': 1, 'school': 2, 'to': 3, 'pizza': 4, 'i': 5, 'go': 6}
문제 1-3
결과값: [[5, 6, 3, 2], [5, 1, 4]]


# 문제2

### **문제 2) TfidfVectorizer 생성하기**

**2-1. `fit()`**

입력 문장들을 이용해 IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: IDF 행렬은 list 형태입니다.
    - ex) [토큰1에 대한 IDF 값, 토큰2에 대한 IDF 값, .... ]
- 조건 2: IDF 값은 아래 식을 이용해 구합니다.
    
    $$
    idf(d,t)=log_e(\frac{n}{1+df(d,t)})
    $$
    
    - $df(d,t)$ : 단어 t가 포함된 문장 d의 개수
    - $n$ : 입력된 전체 문장 개수
- 조건 3: 입력된 문장의 토큰화에는 문제 1에서 만든 Tokenizer를 사용합니다.
    
    

**2-2. `transform()`**

입력 문장들을 이용해 TF-IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output : nested list 형태입니다.
    
    ex) [[tf-idf(1, 1), tf-idf(1, 2), tf-idf(1, 3)], [tf-idf(2, 1), tf-idf(2, 2), tf-idf(2, 3)]]
    
    |  | 토큰1 | 토큰2 | 토큰3 |
    | --- | --- | --- | --- |
    | 문장1 | tf-idf(1,1) | tf-idf(1,2) | tf-idf(1,3) |
    | 문장2 | tf-idf(2,1) | tf-idf(2,2) | tf-idf(2,3) |
- 조건1 : 입력 문장을 이용해 TF 행렬을 만드세요.
    - $tf(d, t)$ : 문장 d에 단어 t가 나타난 횟수
- 조건2 : 문제 2-1( `fit()`)에서 만든 IDF 행렬과 아래 식을 이용해 TF-IDF 행렬을 만드세요
    
    $$
    tf-idf(d,t) = tf(d,t) \times idf(d,t)
    $$

# IDF 구현하기

DF는 문서빈도 Document Frequency
이 값의 역수를 IDF라고 한다.

In [4]:
import os
import string
import math
import pandas as pd

# 모듈 선언

In [5]:
cwd = os.getcwd()

with open(f'{cwd}/random_sentences.txt') as f:
    test_set = f.read().splitlines() 
docs = test_set[:5]

In [6]:
def get_word_list(docs):
    result = []
    for doc in docs:
        result += list(set(doc.lower().translate(str.maketrans('', '', string.punctuation)).split(' ')))
    result = set(result)
    return sorted(list(result));

def get_idf(t, docs):
    n = len(docs)
    df = sum([1 for doc in docs if t in doc])
    return math.log(n / (1 + df))
# pythonic
# return math.log(len(docs) / (1 + sum([1 for doc in docs if t in doc])))

word_list = get_word_list(docs)
idf_table = []

for word in word_list:
    value = get_idf(word, docs)
    idf_table.append(value)

idf_table_pd = pd.DataFrame(idf_table, index = word_list, columns = ["IDF"])
print(idf_table_pd)

                  IDF
accept       0.916291
all          0.916291
asking       0.916291
awaited      0.916291
be           0.916291
bridge       0.916291
brought      0.916291
business     0.916291
but          0.916291
change       0.916291
clown        0.916291
doesnt       1.609438
enchantment  0.916291
ever         0.916291
experience   0.916291
for          0.916291
gingerly     0.916291
he           0.000000
her          0.510826
his          0.916291
ideas        0.916291
irony        0.916291
is           0.223144
it           0.510826
knowing      0.916291
light        0.916291
like         0.916291
list         0.916291
me           0.510826
my           0.916291
neardeath    1.609438
new          0.916291
no           0.510826
on           0.223144
onto         0.916291
other        0.916291
saw          0.916291
she          0.916291
side         0.916291
sound        0.916291
stepped      0.916291
suit         0.916291
that         0.510826
the          0.916291
to        

## TF 구현하기
2-2 framsform()의 
- 조건1 : 입력 문장을 이용해 TF 행렬을 만드세요.
    - $tf(d, t)$ : 문장 d에 단어 t가 나타난 횟수
    
를 구해보겠습니다

In [56]:
tf_table = []

for doc in docs:
    row = []
    for word in word_list:
        row.append(doc.count(word))
    else:
        tf_table.append(row)

tf_table_result = pd.DataFrame(tf_table, columns = word_list)
print(tf_table_result)

   accept  all  asking  awaited  be  bridge  brought  business  but  change  \
0       1    0       1        0   0       0        0         0    1       1   
1       0    1       0        0   0       0        0         1    0       0   
2       0    0       0        1   0       1        0         0    0       0   
3       0    0       0        0   1       0        0         0    0       0   
4       0    0       0        0   0       0        1         0    0       0   

   ...  that  the  to  travel  wanting  was  when  who  will  wore  
0  ...     0    0   2       0        1    0     0    1     0     0  
1  ...     0    0   0       0        0    1     1    0     0     1  
2  ...     1    3   1       0        0    0     0    0     0     0  
3  ...     1    0   0       1        0    0     0    0     1     0  
4  ...     0    0   1       0        0    0     0    0     0     0  

[5 rows x 52 columns]


# 문제풀이

In [14]:
class TfidfVectorizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
    self.fit_checker = False
    self.idf_mat = []
  
  def fit(self, sequences):
    tokenized = self.tokenizer.fit_transform(sequences)
    for key, value in self.tokenizer.word_dict.items():
        new_value = get_idf(value, tokenized)
        self.idf_mat.append(new_value)
#     테이블로보기
#     idf_table_pd = pd.DataFrame(self.idf_mat, index = list(self.tokenizer.word_dict.values()), columns = ["IDF"])
#     print(idf_table_pd)
    self.fit_checker = True
    

  def transform(self, sequences):
    if self.fit_checker:
      tokenized = self.tokenizer.transform(sequences)
      '''
      문제 2-2.
      '''
      return self.tfidf_matrix
    else:
      raise Exception("TfidfVectorizer instance is not fitted yet.")

  
  def fit_transform(self, sequences):
    self.fit(sequences)
    return self.transform(sequences)
  def get_idf(word, seq):
        word_dict = self.tokenizer.word_dict 
        target_token = word_dict[word]
        value = 0
        for s in seq:
            if target_token in s:
                value += 1;
            else:
                continue
            
        result = len(seq) / (1 + value)
        pass
    # return math.log(len(docs) / (1 + sum([1 for doc in docs if t in doc])))

In [13]:
tokenizer = Tokenizer()
tf_idf = TfidfVectorizer(tokenizer)
tf_idf.fit(docs)



fitted word dict: {'oov': 0, 'clown': 1, 'travel': 2, 'doesnt': 3, 'wanting': 4, 'side': 5, 'me': 6, 'onto': 7, 'new': 8, 'like': 9, 'be': 10, 'who': 11, 'stepped': 12, 'accept': 13, 'knowing': 14, 'ever': 15, 'business': 16, 'wore': 17, 'awaited': 18, 'it': 19, 'all': 20, 'she': 21, 'other': 22, 'change': 23, 'but': 24, 'my': 25, 'experience': 26, 'brought': 27, 'is': 28, 'her': 29, 'suit': 30, 'gingerly': 31, 'neardeath': 32, 'enchantment': 33, 'irony': 34, 'to': 35, 'ideas': 36, 'light': 37, 'asking': 38, 'he': 39, 'sound': 40, 'the': 41, 'that': 42, 'on': 43, 'his': 44, 'no': 45, 'for': 46, 'will': 47, 'list': 48, 'saw': 49, 'was': 50, 'bridge': 51, 'when': 52}
         IDF
0   1.609438
1   0.916291
2   0.916291
3   0.916291
4   0.916291
5   0.916291
6   0.916291
7   0.916291
8   0.916291
9   0.916291
10  0.916291
11  0.916291
12  0.916291
13  0.916291
14  0.916291
15  0.916291
16  0.916291
17  0.916291
18  0.916291
19  0.916291
20  0.916291
21  0.916291
22  0.916291
23  0.916291
2